In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
from py2neo import Node, Relationship, Graph, Subgraph, NodeMatcher

from req_analysis import Model, Requirement, ContextCluster

# Analysis

In [3]:
graph = Graph(bolt='bolt://localhost:7687/db/data', password="password")

In [4]:
time1 = time.time()
model = Model(graph, 0.015)
print('exec time: ', time.time()-time1)

exec time:  3.2898178100585938


### Example with a single Requirement

<b>Requirement of focus:</b> <i>In the TMT Model, Telescope Control System is a block, M3 System is a package with no content</i>\
[REQ-1-OAD-2055]</strong> The M3 System shall receive and execute real time tilt and rotation commands issued by the Telescope Control System.\


Removes ReqChunks and ReqPhrases from graph

In [34]:
graph.run("""OPTIONAL MATCH (n:ReqChunk)
OPTIONAL MATCH (m:ReqPhrase) 
DETACH DELETE n, m""")

Removes Allocations from graph

In [35]:
graph.run("MATCH ()-[r:ALLOCATED]-() DETACH DELETE r")

In [10]:
data_li = graph.run("Match (n:`TMT Requirement`) WHERE n.req_text<>'' RETURN n LIMIT 1000").data()

In [11]:
matcher = NodeMatcher(graph)
nodematcher = matcher.match("TMT Requirement")

Requirement to focus on: 'b8d75b3f-f5cb-4523-85d6-e4db1aaa0a42'

In [12]:
time1 = time.time()

alloc_list = []

for req_node in data_li[500:800]:
    time2 = time.time()
    req_node = req_node['n']
    req_object = model.create_req_object(req_node)
    req_object.init_req_nodes()
    allocated_context = req_object.context_cluster_mapping()
   
    if allocated_context!=None:
        rel = Relationship(req_node, "ALLOCATED", allocated_context.source_node, weight=1)
        graph.create(rel)


        print('\nREQ TEXT:\n\t', req_node['req_text'])
        print('ALLOCATED NODE:\n\t---', allocated_context.source_node['name'], '\nALLOCATED NODE ATTRIBUTES: ')
        for el in allocated_context.context:
            print('\t-', el['name'])
        print('\nExecution time: ', time.time()-time2)
        print('________________\n')
        

        req_object.match_req_tokens(pos_list=["NOUN", "PROPN", "NUM"], relationship_filter='', label_filter='-ReqPhrase|-TMT Requirement|-DefaultValue', min_level=0, max_level=6, weighted_depth_limit=6)

        model.append_req_graph(req_object)
        model.link_matches(req_object.transclusion_relations)

print('Total exec time: ', time.time()-time1)

... CONTEXT MATCH:  m m (_3574:Block {name: 'Procedure Executive and Analysis Software', uuid: '94471de9-0ac7-4edc-9b09-f0a87db52e25'})
... CONTEXT MATCH:  m m (_9699:Block {name: 'BS', uuid: 'd772d379-324f-4da8-b642-554deb059a63'})

REQ TEXT:
	 NIRES-R		Cylinder		1			1.5				0.5m inside, on axis		Fed by MIRAO
ALLOCATED NODE:
	--- Procedure Executive and Analysis Software 
ALLOCATED NODE ATTRIBUTES: 
	- Integer
	- Real
	- Boolean
	- Real
	- Integer
	- TipTilt
	- Boolean
	- Real
	- Boolean
	- Integer
	- Real
	- Integer
	- Integer
	- Boolean
	- Real
	- Real
	- Real
	- Real
	- Real
	- Real
	- Real
	- Real
	- m
	- Real
	- Boolean
	- Integer
	- Integer
	- Real
	- Integer
	- Real
	- Integer
	- Real
	- Integer
	- Integer
	- Real
	- Integer
	- Integer
	- Boolean
	- Real
	- Integer
	- Real
	- Integer
	- Real
	- Real
	- Real
	- Real
	- Integer
	- Integer
	- Real
	- Boolean
	- Integer
	- Real
	- Integer
	- Integer
	- CalibrationElevationLookupTable
	- CalibrationElevationLookupTable
	- APS Concept

/usr/local/anaconda3/envs/cref/lib/python3.7/site-packages/nltk/cluster/util.py:133: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (numpy.dot(u, v) / (sqrt(numpy.dot(u, u)) * sqrt(numpy.dot(v, v))))


... SOURCE NODE MATCH:  the atmosphere Atmosphere
... SOURCE NODE MATCH:  the atmosphere Atmosphere

REQ TEXT:
	 <html><body>
<span xmlns="http://www.w3.org/1999/xhtml">The <i>primary input </i>to the system is the light from celestial objects under investigation (i.e. science targets) after it has passed through the atmosphere of the Earth. This light is sometimes known as the science beam (or beams if multiple sub-field-of-views are observed simultaneously). When passing through the atmosphere, the wavefront of the original science beam is distorted by time-variable refractive index changes caused by turbulence at various elevations above the surface of the Earth.<br/>The <i>secondary input </i>to the system is the light from celestial sources located near the science beam on the celestial sphere. These sources are called natural guide stars (NGS) and are used for two purposes: (i) to measure telescope tracking errors and misalignments, as well as (ii) to measure atmospheric wavefron